In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
import maketab as mt

In [45]:
def normalize(train_data):
    mins = np.min(train_data, axis=1, keepdims=True)
    maxs = np.max(train_data, axis=1, keepdims=True)
    normalized_train = (train_data - mins) / (maxs - mins)
    params = {'mins': mins, 'maxs': maxs}   
    return normalized_train, params

def denormalize(normalized_train, params):
    mins = params['mins']
    maxs = params['maxs']
    denormalized_train = normalized_train * (maxs - mins) + mins    
    return denormalized_train

def norm(data):
    min_val = np.min(data)
    max_val = np.max(data)
    normalized_data = (data - min_val) / (max_val - min_val)
    return normalized_data

In [20]:
# Define the MLP class (must match the original definition)
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Input to hidden layer
        self.fc2 = nn.Linear(hidden_size, output_size)  # Hidden to output layer
        self.relu = nn.ReLU()  # Activation function
        self.prelu = nn.PReLU()
        self.tanh = nn.Tanh()

    def forward(self, x):
        out = self.fc1(x)  # Pass through the hidden layer
        #out = self.relu(out)  # Apply ReLU activation
        out = self.tanh(out)
        out = self.fc2(out)  # Pass through the output layer
        return out

In [21]:
# Load the checkpoint
checkpoint = torch.load('bat_model.pth')

# Extract architecture parameters
seq_length = checkpoint['seq_length']
num_layers = checkpoint['num_layers']
starting_point = checkpoint['starting_point']
input_size = checkpoint['input_size']
hidden_size = checkpoint['hidden_size']
output_size = checkpoint['output_size']

# Instantiate the model
model = MLP(input_size, hidden_size, output_size)

# Load the state dictionary
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [52]:
def eval(model, data):
    model.eval()
    with torch.no_grad():
        
        input_data = data[0]
        output_data = data[1]
        time_data = data[2]
        #time_data = t = np.linspace(0, 1, len(data[1]))
        for i in range(starting_point, len(input_data) - seq_length, seq_length):
            x = input_data[i:i + seq_length]
            #t = time_data[i:i + seq_length]
            t = norm(np.arange(0, seq_length)*100)
            y = output_data[i:i + seq_length]
            #print(f"x data: {x}")
            #print(f"t data: {t}")

            slope2, slope, intercept = np.polyfit(t, x, 2)
            mean = np.mean(x)
            std = np.std(x)
            #slope = slope*10**(5)
            #slope2 = slope2*10**(8)
            #std = std*10**(1)
            X = torch.tensor((slope2, slope, intercept, mean, std), dtype=torch.float32)
            #print(f"X = {X}")
            pred = model(X).numpy()
            #reg = 10**(-8)*pred[0]*t**2 + 10**(-5)*pred[1]*t + pred[2] #Unscaling the numbers back
            reg = pred[0]*t**2 + pred[1]*t + pred[2] #Unscaling the numbers back
            print(f"result: {reg[0]}")
            
            plt.plot(t, x, c='blue', label="baterie[V]")
            plt.plot(t, y, c='red', label="realna odhadovana kapacita[%]")
            plt.plot(t, reg, c='green', label="predikovana kapacita [%]")
            if (i <= starting_point): #otherwise it will print more than once
                plt.legend()
            plt.errorbar(t[0], pred[3], pred[4], linestyle='None')
            #plt.scatter(t[0], pred[3], color="black", linewidth=1)
            plt.close()

        #plt.show()

In [53]:
# Load your test data (replace this with your actual data loading code)
def load_data(path_dir):
    t, signal = mt.battery(path_dir)
    tleft = 1 - t / max(t)
    t = np.arange(0, len(signal))*100
    print(t[-1])
    return signal, tleft, t

test_data = load_data('data/21-2-25/')
test_data, test_param = normalize(test_data)

# Evaluate the model on the test data
print("Evaluating on test data...")
eval(model, test_data)

416000
Evaluating on test data...
result: 0.998792827129364
result: 0.9988324046134949
result: 0.9988683462142944
result: 0.9988758563995361
result: 0.9989136457443237
result: 0.9990628361701965
result: 0.9989644289016724
result: 0.9989694952964783
result: 0.998979926109314
result: 0.9989929795265198
result: 0.9990207552909851
result: 0.9990248680114746
result: 0.9990609288215637
result: 0.9990846514701843
result: 0.9990600943565369
result: 0.9990813732147217
result: 0.9990796446800232
result: 0.9990941286087036
result: 0.9991018772125244
result: 0.9991390705108643
result: 0.9991329312324524
result: 0.9991433620452881
result: 0.9991510510444641
result: 0.9991786479949951
result: 0.9991710782051086
result: 0.9991804957389832
result: 0.9991698265075684
result: 0.9992177486419678
result: 0.9992771744728088
result: 0.9992607235908508
result: 0.9992530345916748
result: 0.9992236495018005
result: 0.9992183446884155
result: 0.9992282390594482
result: 0.9992433786392212
result: 0.9992470145225